In [33]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import mediapy as media
import zarr
from IPython.display import clear_output
clear_output()

In [1]:

# defining xml paths for the models. 
xml_aloha2 = 'aloha2/sim_cube_transfer.xml'
model = mujoco.MjModel.from_xml_path(xml_aloha2)
data = mujoco.MjData(model)

In [10]:
# loading actions for simulation

actions_zarr = zarr.open('actions.zarr',mode='r')
actions = actions_zarr['action']
actions.shape

(400, 14)

In [34]:
video = np.array([[[255, 0, 0]] * 300] * 300,dtype=np.uint16)
video = [video]*10
print(video[0].shape)
media.show_video(video, fps=2)
 # Shows a red square

(300, 300, 3)


"<source src=""data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAAA5VtZGF0AAACfgYF//963EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE1NSByMjkxNyAwYTg0ZDk4IC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAxOCAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTkgbG9va2FoZWFkX3RocmVhZHM9MSBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxhY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHlyYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3ZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTIgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjPWNxcCBtYnRyZWU9MCBxcD0yMCBpcF9yYXRpbz0xLjQwIHBiX3JhdGlvPTEuMzAgYXE9MACAAAAAgWWIhAD//vSg+BTQV/N81TRVgVEzVJoZM8r1YgAAAwAAPbj/eawoAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAADAAD5gQAAAApBmiRsf+RAAAJuAAAACUGeQnivAADpgQAAAAkBnmF0Sf8AAQ8AAAAJAZ5jakn/AAEPAAAAEEGaaEmoQWiZTA//5EAAAm8AAAALQZ6GRREtfwAA6YEAAAAJAZ6ldEn/AAEPAAAACQGep2pJ/wABDwAAABBBmqlJqEFsmUwP/+RAAAJuAAADi21vb3YAAABsbXZoZAAAAAAAAAAAAAAAAAAAA+gAABOIAAEAAAEAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAK1dHJhawAAAFx0a2hkAAAAAwAAAAAAAAAAAAAAAQAAAAAAABOIAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAQAAAAAAAAAAAAAAAAAAQAAAAAEsAAABLAAAAAAAJGVkdHMAAAAcZWxzdAAAAAAAAAABAAATiAAAQAAAAQAAAAACLW1kaWEAAAAgbWRoZAAAAAAAAAAAAAAAAAAAQAAAAUAAVcQAAAAAAC1oZGxyAAAAAAAAAAB2aWRlAAAAAAAAAAAAAAAAVmlkZW9IYW5kbGVyAAAAAdhtaW5mAAAAFHZtaGQAAAABAAAAAAAAAAAAAAAkZGluZgAAABxkcmVmAAAAAAAAAAEAAAAMdXJsIAAAAAEAAAGYc3RibAAAAJhzdHNkAAAAAAAAAAEAAACIYXZjMQAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAEsASwASAAAAEgAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABj//wAAADJhdmNDAWQADP/hABlnZAAMrNlBMJ+7hAAAAwAEAAADABA8UKZYAQAGaOvhssiwAAAAGHN0dHMAAAAAAAAAAQAAAAoAACAAAAAAFHN0c3MAAAAAAAAAAQAAAAEAAABgY3R0cwAAAAAAAAAKAAAAAQAAQAAAAAABAACgAAAAAAEAAEAAAAAAAQAAAAAAAAABAAAgAAAAAAEAAKAAAAAAAQAAQAAAAAABAAAAAAAAAAEAACAAAAAAAQAAQAAAAAAcc3RzYwAAAAAAAAABAAAAAQAAAAoAAAABAAAAPHN0c3oAAAAAAAAAAAAAAAoAAAMHAAAADgAAAA0AAAANAAAADQAAABQAAAAPAAAADQAAAA0AAAAUAAAAFHN0Y28AAAAAAAAAAQAAADAAAABidWR0YQAAAFptZXRhAAAAAAAAACFoZGxyAAAAAAAAAABtZGlyYXBwbAAAAAAAAAAAAAAAAC1pbHN0AAAAJal0b28AAAAdZGF0YQAAAAEAAAAATGF2ZjU4LjI5LjEwMA=="" type=""video/mp4""/> This browser does not support the video tag."


In [14]:

# joint space control of the robot arms 

aloha2_gripper_value = lambda x: 0.0076 + (0.041 - 0.0076)*x 
def set_action(data,action):
        left_gripper,right_gripper = action[6],action[13]

        new_action = np.zeros((16))
        
        # updated left arm position 
        new_action[0:6] = action[0:6]
        new_action[6:8] = [aloha2_gripper_value(left_gripper)]*2
        
        # updated right gripper positioon
        new_action[8:14] = action[7:13]
        new_action[14:16] = [aloha2_gripper_value(right_gripper)]*2

        data.qpos[0:16] = new_action

In [17]:
# simulation 
max_timesteps = len(actions)
frame_rate = 60
height,width = 480,640
frames = []
with mujoco.Renderer(model, height,width) as renderer:
  for i in range(max_timesteps):
    mujoco.mj_step(model,data)
    set_action(data=data,action=actions[i])
    
    renderer.update_scene(data,camera='teleoperator_pov')

    img = renderer.render()
    frames.append(img)

media.show_video(frames,fps =frame_rate)